In [1]:
import pygame, sys, random
import numpy as np
import cPickle as pickle
import tensorflow as tf
import math
import matplotlib.pyplot as plt

In [2]:
input_size = 22
output_size = 8
hidden_size = 10
learning_rate = 0.1
gamma = 0.99
decay_rate = 0.99
score_queue_size = 100
resume = False
D = 3

In [ ]:
X = tf.placeholder(tf.float32,  [None, input_size], name = "input_x")
W1 = tf.get_variable("W1", shape = [input_size, hidden_size], initializer = tf.contrib.layers.xavier_initializer())
layer1 = tf.nn.tanh(tf.matmul(X, W1))
W2 = tf.get_variable("W2", shape = [hidden_size, output_size], initializer = tf.contrib.layers.xavier_initializer())

Qpred = tf.matmul(layer1, W2)

Y = tf.placeholder(shape  = [None, output_size], dtype = tf.float32)

loss = tf.reduce_sum(tf.square(Y-Qpred))
train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

In [3]:
def ckWall(xPos, yPos):
    flagWall = 0
    if(xPos < 400):
        xPos = 400
        flagWall = -1
    elif(xPos > 880):
        xPos = 880
        flagWall = -1
    if(yPos > 690):
        yPos = 690
        flagWall = -1
    elif(yPos < 210):
        yPos = 210
        flagWall = -1
        
    return [xPos, yPos, flagWall]

In [ ]:
def stateGenerator(obsPosition, agtPosition, wallcheck):
    returnSum = []
    for i in range(0,10):
        #returnSum = returnSum + [math.sqrt((agtPosition[0] - obsPosition[i][0])**2 + (agtPosition[1] - obsPosition[i][1])**2)]
        returnSum = returnSum + [agtPosition[0] - obsPosition[i][0], agtPosition[1] - obsPosition[i][1]]
    returnSum = returnSum + [agtPosition[0] - 640, agtPosition[1] - 450]
    return returnSum

In [ ]:
#display
pygame.init()
screen = pygame.display.set_mode([1280,960])
screen.fill([200, 200, 200])

num_episodes = 10000
dis = 0.9
rList = []

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    obstaclePos = [[0, 0] for _ in range(10)]
    for i in range(0,10):
        while True:
            obstaclePos[i][0] = 400 + random.randrange(1,230)
            obstaclePos[i][1] = 690 - random.randrange(1,230)
            if obstaclePos[i][0] <= 620 or obstaclePos[i][0] >= 660:
                if obstaclePos[i][1] >= 470 or obstaclePos[i][1] <= 430:
                    break
                    
    for episodes in range(num_episodes):
        
        x = 400
        y = 690
        
        s = stateGenerator(obstaclePos, [x,y], -1)
        e = 1. / ((episodes/10) + 1)
#         e = 0.1
        rAll = 0
        step_count = 0
        ObjectIndex = -1
        while True:
            step_count += 1
            Qinput = np.reshape(s, [1, input_size])
            Qs = sess.run(Qpred, feed_dict = {X: Qinput})
            xMove = 0
            yMove = 0
            if np.random.rand(1) < e:
                a = random.randrange(0,8)
            else:
                a = np.argmax(Qs)
    
            if a == 0:
                xMove = 1
            elif a == 1:
                xMove = 1
                yMove = 1
            elif a == 2:
                yMove = 1
            elif a == 3:
                xMove = -1
                yMove = 1
            elif a == 4:
                xMove = -1
            elif a == 5:
                xMove = -1
                yMove = -1
            elif a == 6:
                yMove = -1
            elif a == 7:
                xMove = 1
                yMove = -1
                            
            x = x + xMove
            y = y + yMove
            
            wallFlag = 0
            collisionFlag = 0
            [x, y, wallFlag] = ckWall(x, y)
            pygame.draw.circle(screen, [100,100,255],[x,y],10,0)

            ## Draw Environment (Obstacle + Map)
            for i in range(0,10):
#                 obstaclePos[i][0] = obstaclePos[i][0] + random.randrange(-1,2)
#                 obstaclePos[i][1] = obstaclePos[i][1] + random.randrange(-1,2)
#                 [obstaclePos[i][0], obstaclePos[i][1], _] = ckWall(obstaclePos[i][0], obstaclePos[i][1])
#                 if obstaclePos[i][0] == 621:
#                     obstaclePos[i][0] = 620
#                 elif obstaclePos[i][0] == 639:
#                     obstaclePos[i][0] = 640
                    
#                 if obstaclePos[i][1] == 431:
#                     obstaclePos[i][1] = 430
#                 elif obstaclePos[i][1] == 469:
#                     obstaclePos[i][1] = 470
                
                pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], 10, 0)
                if math.sqrt((x - obstaclePos[i][0])**2 + (y - obstaclePos[i][1])**2) <= 20:
                    print("Collision!")
                    collisionFlag = -1
                    ObjectIndex = i
            s1 = stateGenerator(obstaclePos, [x,y], wallFlag)
            
            # more close to the goal, more reward agent can get
            distToGoal = math.sqrt((x - 640)**2 + (y - 450)**2)
            # more close to the obstacle, less reward agent can get
            distToObs = [0 for _ in range(10)]
            for i in range(0,10):
                distToObs[i] = math.sqrt((x - obstaclePos[i][0])**2 + (y - obstaclePos[i][1])**2)
            if distToGoal == 0 or 1/distToGoal >= 100000:
                reward = 100000
            else:
                reward = 10000/distToGoal
                for i in range(0,10):
                    reward -= 500/distToObs[i]

            if(math.sqrt((x - 640)**2 + (y - 450)**2) <= 20):
                print("Goal Reached!")
                collisionFlag = 1
            
            if collisionFlag == -1 or wallFlag == -1:
                Qs[0, a] = -10000
                rList.append(0)
            elif collisionFlag == 1:
                Qs[0, a] = 100000
                rList.append(1)
            else:
                Qinput1 = np.reshape(s1, [1, input_size])
                Qs1 = sess.run(Qpred, feed_dict = {X: Qinput1})
                Qs[0,a] = reward + dis * np.max(Qs1)
            
            sess.run(train, feed_dict={X: Qinput, Y: Qs})
            s = s1
            
#             if episodes % 100 == 0:
#                 print(Qs)
#                 print reward
            if collisionFlag == 1 or collisionFlag == -1:
                break
            pygame.draw.circle(screen, [100,255,100], [640, 450], 20, 1)
            pygame.draw.rect(screen, [255,100,100],[390,200,500,500],1)
            pygame.display.flip()
            screen.fill([200,200,200])
        print("Episode : {}, Object Index: {}".format(episodes, ObjectIndex))

Collision!
Episode : 0, Object Index: 8
Collision!
Episode : 1, Object Index: 8
Collision!
Episode : 2, Object Index: 8
Collision!
Episode : 3, Object Index: 8
Collision!
Episode : 4, Object Index: 8
Collision!
Episode : 5, Object Index: 8
Collision!
Episode : 6, Object Index: 8
Collision!
Episode : 7, Object Index: 8
Collision!
Episode : 8, Object Index: 8
Collision!
Episode : 9, Object Index: 8
Collision!
Episode : 10, Object Index: 8
Collision!
Episode : 11, Object Index: 8
Collision!
Episode : 12, Object Index: 8
Collision!
Episode : 13, Object Index: 8
Collision!
Episode : 14, Object Index: 8
Collision!
Episode : 15, Object Index: 7
Collision!
Episode : 16, Object Index: 8
Collision!
Episode : 17, Object Index: 8
Collision!
Episode : 18, Object Index: 5
Collision!
Episode : 19, Object Index: 1
Collision!
Episode : 20, Object Index: 5
Collision!
Episode : 21, Object Index: 8
Collision!
Episode : 22, Object Index: 8
Collision!
Episode : 23, Object Index: 8
Collision!
Episode : 24, O

Collision!
Episode : 200, Object Index: 8
Collision!
Episode : 201, Object Index: 2
Collision!
Episode : 202, Object Index: 8
Collision!
Episode : 203, Object Index: 8
Collision!
Episode : 204, Object Index: 8
Collision!
Episode : 205, Object Index: 2
Collision!
Episode : 206, Object Index: 8
Collision!
Episode : 207, Object Index: 8
Collision!
Episode : 208, Object Index: 8
Collision!
Episode : 209, Object Index: 8
Collision!
Episode : 210, Object Index: 8
Collision!
Episode : 211, Object Index: 8
Collision!
Episode : 212, Object Index: 0
Collision!
Episode : 213, Object Index: 8
Collision!
Episode : 214, Object Index: 8
Collision!
Episode : 215, Object Index: 8
Collision!
Episode : 216, Object Index: 8
Collision!
Episode : 217, Object Index: 8
Collision!
Episode : 218, Object Index: 3
Collision!
Episode : 219, Object Index: 8
Collision!
Episode : 220, Object Index: 8
Collision!
Episode : 221, Object Index: 8
Collision!
Episode : 222, Object Index: 8
Collision!
Episode : 223, Object I

Collision!
Episode : 396, Object Index: 8
Collision!
Episode : 397, Object Index: 8
Collision!
Episode : 398, Object Index: 8
Collision!
Episode : 399, Object Index: 8
Collision!
Episode : 400, Object Index: 8
Collision!
Episode : 401, Object Index: 8
Collision!
Episode : 402, Object Index: 8
Collision!
Episode : 403, Object Index: 8
Collision!
Episode : 404, Object Index: 8
Collision!
Episode : 405, Object Index: 8
Collision!
Episode : 406, Object Index: 8
Collision!
Episode : 407, Object Index: 3
Collision!
Episode : 408, Object Index: 8
Collision!
Episode : 409, Object Index: 8
Collision!
Episode : 410, Object Index: 8
Collision!
Episode : 411, Object Index: 8
Collision!
Episode : 412, Object Index: 8
Collision!
Episode : 413, Object Index: 8
Collision!
Episode : 414, Object Index: 8
Collision!
Episode : 415, Object Index: 8
Collision!
Episode : 416, Object Index: 8
Collision!
Episode : 417, Object Index: 8
Collision!
Episode : 418, Object Index: 8
Collision!
Episode : 419, Object I

Collision!
Episode : 591, Object Index: 8
Collision!
Episode : 592, Object Index: 8
Collision!
Episode : 593, Object Index: 8
Collision!
Episode : 594, Object Index: 8
Collision!
Episode : 595, Object Index: 8
Collision!
Episode : 596, Object Index: 8
Collision!
Episode : 597, Object Index: 8
Collision!
Episode : 598, Object Index: 8
Collision!
Episode : 599, Object Index: 8
Collision!
Episode : 600, Object Index: 8
Collision!
Episode : 601, Object Index: 8
Collision!
Episode : 602, Object Index: 8
Collision!
Episode : 603, Object Index: 8
Collision!
Episode : 604, Object Index: 8
Collision!
Episode : 605, Object Index: 8
Collision!
Episode : 606, Object Index: 8
Collision!
Episode : 607, Object Index: 8
Collision!
Episode : 608, Object Index: 8
Collision!
Episode : 609, Object Index: 8
Collision!
Episode : 610, Object Index: 8
Collision!
Episode : 611, Object Index: 8
Collision!
Episode : 612, Object Index: 8
Collision!
Episode : 613, Object Index: 8
Collision!
Episode : 614, Object I

Collision!
Episode : 788, Object Index: 8
Collision!
Episode : 789, Object Index: 8
Collision!
Episode : 790, Object Index: 8
Collision!
Episode : 791, Object Index: 8
Collision!
Episode : 792, Object Index: 8
Collision!
Episode : 793, Object Index: 8
Collision!
Episode : 794, Object Index: 8
Collision!
Episode : 795, Object Index: 8
Collision!
Episode : 796, Object Index: 0
Collision!
Episode : 797, Object Index: 8
Collision!
Episode : 798, Object Index: 8
Collision!
Episode : 799, Object Index: 8
Collision!
Episode : 800, Object Index: 8
Collision!
Episode : 801, Object Index: 8
Collision!
Episode : 802, Object Index: 8
Collision!
Episode : 803, Object Index: 8
Collision!
Episode : 804, Object Index: 8
Collision!
Episode : 805, Object Index: 8
Collision!
Episode : 806, Object Index: 8
Collision!
Episode : 807, Object Index: 8
Collision!
Episode : 808, Object Index: 8
Collision!
Episode : 809, Object Index: 8
Collision!
Episode : 810, Object Index: 8
Collision!
Episode : 811, Object I

...

Collision!
Episode : 1176, Object Index: 8
Collision!
Episode : 1177, Object Index: 8
Collision!
Episode : 1178, Object Index: 8
Collision!
Episode : 1179, Object Index: 8
Collision!
Episode : 1180, Object Index: 8
Collision!
Episode : 1181, Object Index: 8
Collision!
Episode : 1182, Object Index: 8
Collision!
Episode : 1183, Object Index: 8
Collision!
Episode : 1184, Object Index: 8
Collision!
Episode : 1185, Object Index: 8
Collision!
Episode : 1186, Object Index: 8
Collision!
Episode : 1187, Object Index: 8
Collision!
Episode : 1188, Object Index: 8
Collision!
Episode : 1189, Object Index: 8
Collision!
Episode : 1190, Object Index: 8
Collision!
Episode : 1191, Object Index: 8
Collision!
Episode : 1192, Object Index: 8
Collision!
Episode : 1193, Object Index: 8
Collision!
Episode : 1194, Object Index: 8
Collision!
Episode : 1195, Object Index: 8
Collision!
Episode : 1196, Object Index: 8
Collision!
Episode : 1197, Object Index: 8
Collision!
Episode : 1198, Object Index: 8
Collision!


Collision!
Episode : 1368, Object Index: 8
Collision!
Episode : 1369, Object Index: 8
Collision!
Episode : 1370, Object Index: 8
Collision!
Episode : 1371, Object Index: 8
Collision!
Episode : 1372, Object Index: 8
Collision!
Episode : 1373, Object Index: 8
Collision!
Episode : 1374, Object Index: 8
Collision!
Episode : 1375, Object Index: 8
Collision!
Episode : 1376, Object Index: 8
Collision!
Episode : 1377, Object Index: 8
Collision!
Episode : 1378, Object Index: 8
Collision!
Episode : 1379, Object Index: 8
Collision!
Episode : 1380, Object Index: 8
Collision!
Episode : 1381, Object Index: 8
Collision!
Episode : 1382, Object Index: 8
Collision!
Episode : 1383, Object Index: 8
Collision!
Episode : 1384, Object Index: 8
Collision!
Episode : 1385, Object Index: 8
Collision!
Episode : 1386, Object Index: 8
Collision!
Episode : 1387, Object Index: 8
Collision!
Episode : 1388, Object Index: 8
Collision!
Episode : 1389, Object Index: 8
Collision!
Episode : 1390, Object Index: 8
Collision!


Goal Reached!
Episode : 1547, Object Index: -1
Goal Reached!
Episode : 1548, Object Index: -1
Goal Reached!
Episode : 1549, Object Index: -1
Goal Reached!
Episode : 1550, Object Index: -1
Goal Reached!
Episode : 1551, Object Index: -1
Goal Reached!
Episode : 1552, Object Index: -1
Goal Reached!
Episode : 1553, Object Index: -1
Goal Reached!
Episode : 1554, Object Index: -1
Goal Reached!
Episode : 1555, Object Index: -1
Collision!
Episode : 1556, Object Index: 0
Collision!
Episode : 1557, Object Index: 0
Collision!
Episode : 1558, Object Index: 4
Collision!
Episode : 1559, Object Index: 7
Collision!
Episode : 1560, Object Index: 7
Collision!
Episode : 1561, Object Index: 7
Collision!
Episode : 1562, Object Index: 7
Collision!
Episode : 1563, Object Index: 8
Collision!
Episode : 1564, Object Index: 8
Collision!
Episode : 1565, Object Index: 8
Collision!
Episode : 1566, Object Index: 7
Collision!
Episode : 1567, Object Index: 6
Collision!
Episode : 1568, Object Index: 8
Collision!
Episode

Collision!
Episode : 1737, Object Index: 7
Collision!
Episode : 1738, Object Index: 7
Collision!
Episode : 1739, Object Index: 7
Goal Reached!
Episode : 1740, Object Index: -1
Collision!
Episode : 1741, Object Index: 7
Collision!
Episode : 1742, Object Index: 7
Collision!
Episode : 1743, Object Index: 7
Collision!
Episode : 1744, Object Index: 7
Collision!
Episode : 1745, Object Index: 7
Collision!
Episode : 1746, Object Index: 7
Collision!
Episode : 1747, Object Index: 7
Collision!
Episode : 1748, Object Index: 7
Collision!
Episode : 1749, Object Index: 7
Collision!
Episode : 1750, Object Index: 7
Collision!
Episode : 1751, Object Index: 7
Collision!
Episode : 1752, Object Index: 7
Collision!
Episode : 1753, Object Index: 7
Collision!
Episode : 1754, Object Index: 7
Collision!
Episode : 1755, Object Index: 7
Collision!
Episode : 1756, Object Index: 7
Collision!
Episode : 1757, Object Index: 7
Collision!
Episode : 1758, Object Index: 8
Collision!
Episode : 1759, Object Index: 8
Collisi

Collision!
Episode : 1929, Object Index: 8
Collision!
Episode : 1930, Object Index: 8
Collision!
Episode : 1931, Object Index: 8
Collision!
Episode : 1932, Object Index: 8
Collision!
Episode : 1933, Object Index: 4
Collision!
Episode : 1934, Object Index: 8
Collision!
Episode : 1935, Object Index: 8
Collision!
Episode : 1936, Object Index: 8
Collision!
Episode : 1937, Object Index: 8
Collision!
Episode : 1938, Object Index: 5
Collision!
Episode : 1939, Object Index: 5
Collision!
Episode : 1940, Object Index: 8
Collision!
Episode : 1941, Object Index: 5
Collision!
Episode : 1942, Object Index: 4
Collision!
Episode : 1943, Object Index: 8
Collision!
Episode : 1944, Object Index: 8
Collision!
Episode : 1945, Object Index: 8
Collision!
Episode : 1946, Object Index: 5
Goal Reached!
Episode : 1947, Object Index: -1
Collision!
Episode : 1948, Object Index: 8
Collision!
Episode : 1949, Object Index: 8
Collision!
Episode : 1950, Object Index: 8
Collision!
Episode : 1951, Object Index: 8
Collisi

In [ ]:
print("Percent of successful episodes: " + str(sum(rList)/num_episodes) + "%")
plt.bar(range(len(rList)), rList, color = "white", width = 0.00001)
plt.show()